In [9]:
!pip install dadmatools spacy_stanza hazm ipywidgets 

### Read symbols

In [40]:
import json

def create_output(output_type, marker, span):
    return {"type":output_type, "marker":marker, "span":repr(span)}

class Symbolizer:
    def __init__(self):
        self.symbols = []
        with open('namad.txt') as symbols_file:
            line = symbols_file.readline()
            while line:
                n = line.split('\t')
                self.symbols.append(n[0])
                # TODO: we can add more information to a symbol here
                # namad_description.append(n[1])
                line = symbols_file.readline()

        with open('symbols.json', 'r') as file:
            symbols = json.load(file)['symbols']
            
        len(symbols)

    def symbolize(self, sentence):
        output = []
        for namad in self.symbols:
            for m in re.finditer(namad, sentence):
                output.append(create_output("نماد", namad, m.span()))
        return output

        expression = "|".join(symbols)



In [10]:
import stanza
import spacy_stanza

stanza.install_corenlp()
stanza.download('fa')
stanza_nlp = spacy_stanza.load_pipeline("fa")

2022-04-14 07:12:27 WARNING: Directory /root/stanza_corenlp already exists. Please install CoreNLP to a new directory.


2022-04-14 07:12:27 INFO: Downloading default packages for language: fa (Persian)...
2022-04-14 07:12:28 INFO: File exists: /root/stanza_resources/fa/default.zip.
2022-04-14 07:12:30 INFO: Finished downloading models and saved to /root/stanza_resources.
2022-04-14 07:12:30 INFO: Loading these models for language: fa (Persian):
| Processor | Package |
-----------------------
| tokenize  | perdt   |
| mwt       | perdt   |
| pos       | perdt   |
| lemma     | perdt   |
| depparse  | perdt   |

2022-04-14 07:12:30 INFO: Use device: cpu
2022-04-14 07:12:30 INFO: Loading: tokenize
2022-04-14 07:12:30 INFO: Loading: mwt
2022-04-14 07:12:30 INFO: Loading: pos
2022-04-14 07:12:31 INFO: Loading: lemma
2022-04-14 07:12:31 INFO: Loading: depparse
2022-04-14 07:12:31 INFO: Done loading processors!


In [55]:
texts = [
    # "برکت همین افشای ب باعث می شود سهم سه درصد مثبت بشود. بخاطر همین میگم پیگیر باشید.",
    # "نماد برکت امروز عرضه اولیه خیلی خوبی داره.",
    # 'روز چهارشنبه یک دفعه برای خودشون افشا زدن.',
    # "سهام وغدیر و خزر کاهش یافت.",
    # "برای خودشون ی افشایی زدن.",
    # "آ س پ امروز بالا رفت.",
    # "ریزش بازار به دلیل حمله‌ی روسیه هست.",
    # "فک کنم یه اصلاح قیمتی و کمی ریزش داشته باشیم.",
    "قرارداد با آمریکا باعث افت قیمت سهم وغدیر شد",
    # "یک نکته‌ی تکنیکالی هم در صورت دستکاری نشدن اضافه کنم، کندلی که روز سه شنبه‌ی گذشته ثبت کرد کامل است",
    # "روز چهارشنبه یه دفعه برای خودشون افشا زدن",
    # "رشد قیمت‌ها باعث ایجاد صف خرید در سهم پرشیا شد",
    # "شاخص به ۲ میلیون می‌رسه",
    # "آمریکا باعث ریزش بازار شد",
    # "آمریکا موجب ریزش بازار شد",
    # "آمریکا دلیل ریزش بازار شد",
    # "کاهش قیمت سهم عجیب بود",
    # "قیمت زیاد شد",
    # "قیمت زیاد است",
    # "به کتابخانه رفتم.",
    # "به کتابخانه رفت.",
    # "پول در جیب من است.",
    # "سهم قیمتش پایین است",
    # "حضور تو موجب خوشحالی من در هوای بارانی است",
]
correct_sentences = [
    "ارزش سهام شرکت نفت کاهش یافت",
    "قرارداد با آمریکا باعث افت قیمت سهم وغدیر شد",
    "ریزش بازار به دلیل حمله‌ی روسیه است.",
    "فک کنم یه اصلاح قیمتی و کمی ریزش داشته باشیم.",
    "یک نکته‌ی تکنیکالی هم در صورت دستکاری نشدن اضافه کنم، کندلی که روز سه شنبه‌ی گذشته ثبت کرد کامل است",
    "روز چهارشنبه یه دفعه برای خودشون افشا زدن",
    "رشد قیمت‌ها باعث ایجاد صف خرید در سهم پرشیا شد",
    "شاخص به ۲ میلیون می‌رسه",
    "آمریکا باعث ریزش بازار شد",
    "آمریکا موجب ریزش بازار شد",
    "آمریکا دلیل ریزش بازار شد",
    "کاهش قیمت سهم عجیب بود",
    "قیمت زیاد شد",
    "قیمت زیاد است",
    "سهم قیمتش پایین است",
    "حضور تو موجب خوشحالی من در هوای بارانی است",
    "برکت همین افشای ب باعث شد سهم سه درصد مثبت شود",
    "بخاطر همین میگم پیگیر باشید",
    "گزارش فعالیت ماهانه دوره ۱ ماهه منتهی به ۱۴۰۰/۰۹/۳۰ برای دیران منتشر شد",
    "نرخ گاز خوراک پتروشیمی‌ها طبق فرمول ابلاغی وزارت نفت و با اعمال سقف نرخ گاز صادراتی، حداکثر پنج هزار تومان تعیین شده است",
    "نرخ گاز مصرفی صنایع سیمان و سایر صنایع معادن ۱۰ درصد نرخ خوراک پتروشیمی‌ها تعیین شده است",
    "کارشناسان می‌گویند تعیین شدن نرخ خوراک پتروشیمی‌ها به سودآوری شرکت‌ها در بلند مدت کمک می‌کند",
    "قیمت دلار بازار آزاد به بیش از ۲۸ هزار تومان رسید. کارشناسان عقیده دارند افزایش مقطعی قیمت دلار، تاثیر منفی روی بازار سرمایه دارد",
    "عرضه اولیه جدید می‌تواند، نقدینگی بیشتری به بازارا سرمایه وارد کند","کارشناسان می‌گویند که اگر ارز ۴۲۰۰ تومانی حذف شود، صنعت دارو در کوتاه مدت نزولی می‌شود",
    "شرکت سرمایه‌گذاری توسعه صنعت و تجارت اعلام کرد که در نظر دارد نسبت به فروش ملک خود واقع در شهر کرمان از طریق مزایده عمومی اقدام کند",
    "کارشناسان می‌گویند کاهش نرخ سود بین بانکی می‌تواند نقدینگی جدید به بورس وارد کند.",
    "کارشناسان می‌گویند که کاهش تامین مالی از بورس می‌تواند این بازار را صعودی و نقدینگی جدید وارد آن کند.",
    "جلسه معارفه آسیاتک روز دوشنبه برگزار می‌شود. (سنا) احتمالا آسیاتک اولین عرضه اولیه بازار سرمایه در قرن جدید باشد.",
    "پذیره‌نویسی نخستین صندوق سهامی با نماد سلام از روز دوشنه در بازار سهام آغاز می‌شود.",
    "کارشناسان معتقدند که پذیره‌نویسی صندوق‌های جدید نقدینگی جریان نقدینگی در بازار را تقویت می‌کند",
    "شرکت پالایش نفت اصفهان با نماد شپنا افزایش سرمایه خود را از محل سود انباشته ثبت کرد.",
    "ولبهمن قرار است سرمایه ۸۰‌ میلیارد تومانی خود را به ۱۰۰ میلیارد تومان برسد.",
]

amin_terms = [
             'قرارداد',
]

terms = [
    'ضرر',
    'سود',
    'اطلاعیه',
    'افزایش سرمایه',
    'تقسیم سود',
    'دامنه نوسان',
    'نوسان شدید',
    'سهم رانتی',
    'عرضه اولیه',
    'افشا',
    'فعالیت ماهانه',
    'فعالیت سالانه',
    'کاهش',
    'افزایش',
]
events = [
    "سود",
    "مثبت",
    "منفی",
    "صعود",
    "نزول",
    "افزایش",
    "کاهش",
    "رشد",
    "ریزش",
    "افشا",
    "اصلاح",
    "افت",
    "سیگنال",
    "تعیین نرخ",
    "عرضه اولیه"
]
reason_propositions = [
    "دلیل",
    "باعث",
    "موجب",
    "برای"
]


In [18]:
# from parsivar import Normalizer as ParsivarNormalizer
from dadmatools.models.normalizer import Normalizer
normalizer = Normalizer(
    full_cleaning=False,
    unify_chars=True,
    refine_punc_spacing=True,
    remove_extra_space=True,
    remove_puncs=False,
    remove_html=False,
    remove_stop_word=False,
    replace_email_with="<EMAIL>",
    replace_number_with=None,
    replace_url_with="<URL",
    replace_mobile_number_with="<MOBILE_NUMBER>",
    replace_emoji_with="<EMOJI>",
    replace_home_number_with="<HOME_NUMBER>"
)

# normalizer2 = ParsivarNormalizer()

In [59]:
import re
import spacy
from spacy import displacy


for index, text in enumerate(correct_sentences):
    print('text is: ', text)
    text = normalizer.normalize(text)
    try:
        
        # doc = dadma_nlp(text)
        doc = stanza_nlp(text)

        # print(spacy.explain('nsubj'))
        # print(spacy.explain('xcomp'))
        # from spacy.symbols import nsubj, VERB
        # verbs = set()
        # for possible_subject in doc:
        #     # print(possible_subject.head.pos)
        #     if possible_subject.dep == nsubj and possible_subject.head.pos == VERB:
        #         print(possible_subject)
        #         verbs.add(possible_subject.head)
        # print(verbs)


        # print(f'sentence {index + 1}: ...')
        # for token in doc:
        #     print(f'word: {token.text:12}, pos: {token.pos_:10}, tag: {token.tag_:10}, dep: {token.dep_:15}')
        #     print("\n")
        
        # it works in dadma
        # sentences = doc._.sentences
        # print(type(doc))
        # print(doc._)
        # print(type(doc._))
        
        # for sentence in sentences:
        #     print('sent is: ', text)
        #     sentence_text = sentence.text
        # for token in sentence:
        for token in doc:
            # print('----------------------------------------')
            # print('token: ', token)
            # token_text = token.text
            # print('token_next: ', token_text)
            # lemma = token.lemma_ ## this has value only if lem is called
            # print('lemma: ', lemma)
            # pos_tag = token.pos_ ## this has value only if pos is called
            # print('pos_tag: ', pos_tag)
            # dep = token.dep_ ## this has value only if dep is called
            # print('dep: ', dep)
            # # dep_arc = token._.dep_arc ## this has value only if dep is called
            # # print('dep_arc', dep_arc)
            # y = ''
            # for x in token.subtree:
            #   y += str(x) + ' '
            # print('subtree: ', y)
            # print('children: ', [child for child in token.children])

            # print(token_text, pos_tag, dep, dep_arc)
            # if token.pos_ == "AUX":
            #     token.pos_ = "VERB"



            for event in events:
              pattern = re.compile(rf'\b{event}\b')
              if not pattern.match(token.text):
                continue
              
              print('---------- event founded! keyword: ', event)
              
              extracted_event = ''
              start = None
              end = None
              for x in token.subtree:
                extracted_event += str(x) + ' '
                # print('---2', x)
                if start is None:
                  start = x.idx
                end = x.idx + len(x.text)

              print('event: ', extracted_event)
              # print(text[start:end])
              # print(token.subtree.span)
              print(create_output('واقعه', extracted_event, (start,end)))

    #       def create_output(output_type, marker, span):
    # return {"type":output_type, "marker":marker, "span":repr(span)}
                
        # sent_constituency = doc._.constituency ## this has value only if cons is called
        # print('sent_constituency', sent_constituency)
        # sent_chunks = doc._.chunks ## this has value only if cons is called
        # print('sent_chunks', sent_chunks)
        # for chunk in doc.noun_chunks:
        #   print(chunk.text, chunk.root.text, chunk.root.dep_,
        #     chunk.root.head.text)

        # ners = doc._.ners ## this has value only if ner is called
        # print(sent_constituency)
        # print(sent_chunks)
        
        print("\n\n\n")
        
        displacy.render(doc, style="dep", jupyter=True, options={'distance': 120,"collapse_phrases" : True})
        # spacy.displacy.serve(doc, style='ent')

        


        
    except Exception as e:
        # print(str(e))
        raise e
        
        # print(f"ERRR {text}")
    
        
    

[W109] Unable to save user hooks while serializing the doc. Re-add any required user hooks to the doc after processing.


text is:  ارزش سهام شرکت نفت کاهش یافت
---------- event founded! keyword:  کاهش
event:  کاهش 
{'type': 'واقعه', 'marker': 'کاهش ', 'span': '(19, 23)'}






text is:  قرارداد با آمریکا باعث افت قیمت سهم وغدیر شد


[W109] Unable to save user hooks while serializing the doc. Re-add any required user hooks to the doc after processing.


---------- event founded! keyword:  افت
event:  افت قیمت سهم وغدیر 
{'type': 'واقعه', 'marker': 'افت قیمت سهم وغدیر ', 'span': '(23, 41)'}






...

text is:  شرکت پالایش نفت اصفهان با نماد شپنا افزایش سرمایه خود را از محل سود انباشته ثبت کرد.


[W109] Unable to save user hooks while serializing the doc. Re-add any required user hooks to the doc after processing.


---------- event founded! keyword:  افزایش
event:  افزایش سرمایه خود را 
{'type': 'واقعه', 'marker': 'افزایش سرمایه خود را ', 'span': '(36, 56)'}
---------- event founded! keyword:  سود
event:  سود انباشته 
{'type': 'واقعه', 'marker': 'سود انباشته ', 'span': '(64, 75)'}






text is:  ولبهمن قرار است سرمایه ۸۰‌ میلیارد تومانی خود را به ۱۰۰ میلیارد تومان برسد.


[W109] Unable to save user hooks while serializing the doc. Re-add any required user hooks to the doc after processing.
